#### Importing Libraries

In [61]:
# 1️⃣ Setup & Imports
import re
import random
import pandas as pd
import json

##### Creating empty dictionary

In [20]:
# Step-by-step creation of the ICU Data Dictionary

# 1️⃣ Start with an empty dictionary
icu_data_dict_granular = {}

##### .2. Build Patterns

##### 1.2. Stress-Test

##### 1.3. Add Identifiers to Dict

## 1. Patient Identifiers

### 1.1. Names (Full Matches)

##### 1.1.2. Build Pattern

## 4. Clinical & Medical Data

### 4.1. Diagnósticos

### 4.2. História Clínica

### 4.3. Exame Físico

### 4.4. Impressão Evolutiva

##### 1.1.2. Build Pattern with live iteraction



https://regex101.com/


In [ ]:
# Name could include the word "paciente", which has no gender inflection, it could just say "nome" or complete name: "nome completo"
Field_Identifier = [r"\b(?:[Nn]ome)\b",
                    r"[nN]ome\s{0,3}[cC]omplete"
                    r"\b[Nn]ome\s{0,3}(?:d[oa])?\s{0,3}[Pp]aciente\b",
                    r"[Pp]aciente"]

# 1. This is a Regex to check for names that are minimally complete: at least one first name and one surname, providing FULL ABSOLUTE MATCHES with the widest range of patterns possible.
# 2. I did this pattern mostly to train and outstretch Regex possibilities, we probably should use a simpler approach to detecting names.
# 3. Names in Brazil can be really creative (differently from Portugal, that has a law for naming, with a determined list of names that could be given at birth), so we must account for many variations in Brazil.
# 4. Added a specificity to Portuguese names which is the presence of possessive particles ("de","do", "da"). Also included the same characteristics from other languages such as  arabian, dutch, italian, german and spanish, since we had considerable immigration from these groups to Brazil."
# 5. Added specific accentuations that correspond to portuguese such as circunflex "^" and tilda "~"
# 6. Added a possible hyphen that in some cases could separate composite names.
# 7. For this field it is really important not to ignore uppercase.

Content_Identifier =  r"\b[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3}-?(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\bvon der\b|\bvan der\b|\bvan den\b|du|\bde la\b|al|bin|ibn|el)?\s{0,3}[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3}-?(?:(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\bvon der\b|\bvan der\b|\bvan den\b|du|\bde la\b|al|bin|ibn|el)?\s{0,3}-?[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3})*\b",



#### 1.1.2. Positive Stress-Test (PST)


In [21]:

# 2️⃣ Add the main category "Patient_Identifiers"
icu_data_dict_granular["Patient_Identifiers"] = {}

# 3️⃣ Add the "Fields" subcategory
icu_data_dict_granular["Patient_Identifiers"]["Fields"] = {}

# 4️⃣ Add the "Complete_Name" field under "Fields"
icu_data_dict_granular["Patient_Identifiers"]["Fields"]["Complete_Name"] = {}


In [ ]:
# Name could include the word "paciente", which has no gender inflection, it could just say "nome" or complete name: "nome completo"
Field_Identifier = [r"\b(?:[Nn]ome)\b",
                    r"[nN]ome\s{0,3}[cC]omplete"
                    r"\b[Nn]ome\s{0,3}(?:d[oa])?\s{0,3}[Pp]aciente\b",
                    r"[Pp]aciente"]

# 1. This is a Regex to check for names that are minimally complete: at least one first name and one surname, providing FULL ABSOLUTE MATCHES with the widest range of patterns possible.
# 2. I did this pattern mostly to train and outstretch Regex possibilities, we probably should use a simpler approach to detecting names.
# 3. Names in Brazil can be really creative (differently from Portugal, that has a law for naming, with a determined list of names that could be given at birth), so we must account for many variations in Brazil.
# 4. Added a specificity to Portuguese names which is the presence of possessive particles ("de","do", "da"). Also included the same characteristics from other languages such as  arabian, dutch, italian, german and spanish, since we had considerable immigration from these groups to Brazil."
# 5. Added specific accentuations that correspond to portuguese such as circunflex "^" and tilda "~"
# 6. Added a possible hyphen that in some cases could separate composite names.
# 7. For this field it is really important not to ignore uppercase.

Content_Identifier =  r"\b[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3}-?(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\bvon der\b|\bvan der\b|\bvan den\b|du|\bde la\b|al|bin|ibn|el)?\s{0,3}[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3}-?(?:(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\bvon der\b|\bvan der\b|\bvan den\b|du|\bde la\b|al|bin|ibn|el)?\s{0,3}-?[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3})*\b",


#### 1.1.2. Positive Stress-Test (PST)

In [23]:
# Define components of Portuguese full names
first_names = ["João", "Maria", "Carlos", "Fernanda", "Antônio", "Ênio", "Beatriz", "Luís", "Ana", "Paulo", "Júlia", "Eduardo"]
middle_names = ["José", "Clara", "Júnior", "Paulo", "Pedro", "Silva", "Jara", "Miguel", "Henrique", "Neves", "Vales"]
last_names = ["Santos", "Costa", "Souza", "Nascimento", "Pereira", "Oliveira", "Vales", "Ribeiro", "Mendes", "Rodrigues"]
particles = ["da", "de", "do", "dos", "das", "di", "von der","al", "ibn", "von"]

# Generate random full names with variation
def generate_portuguese_names(n):
    names = []
    for _ in range(n):
        first = random.choice(first_names)
        middle = random.choice(middle_names) if random.random() > 0.5 else ""
        particle = random.choice(particles) if random.random() > 0.3 else ""
        last = random.choice(last_names)
        hyphen = "-" if random.random() > 0.8 else " "
        
        # Construct name with variations
        full_name = first
        if middle:
            full_name += hyphen + middle if random.random() > 0.7 else " " + middle
        if particle:
            full_name += " " + particle
        full_name += " " + last

        # Add diacritic variations randomly
        full_name = full_name.replace("a", "à") if random.random() > 0.9 else full_name
        full_name = full_name.replace("o", "ô") if random.random() > 0.9 else full_name
        full_name = full_name.replace("e", "ê") if random.random() > 0.9 else full_name

        names.append(full_name)
    return names

# Generate test dataset
test_names = generate_portuguese_names(100)

# Define the regex pattern for full names
name_regex_test = re.compile(
    r"\b[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3}-?(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\bvon der\b|\bvan der\b|\bvan den\b|du|de la|al|bin|ibn|el)?\s{0,3}[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3}-?(?:(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\bvon der\b|\bvan der\b|\bvan den\b|du|\bde la\b|al|bin|ibn|el)?\s{0,3}-?[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3})*\b",

)

# Test regex against generated names
matches_test = [name for name in test_names if name_regex_test.fullmatch(name)]

# Display results
df_results_test = pd.DataFrame({"Generated Name": test_names, "Matched": ["✅" if name in matches_test else "❌" for name in test_names]})


In [24]:
df_results_test #The names that failed are not "real names", so the identifier seems to passed the stress-test.

,Generated Name,Matched
0,Júlia von der Nascimento,✅
1,Maria al Rodrigues,✅
2,Carlos Neves Souza,✅
3,Eduàrdo Jàrà Pereirà,✅
4,Júlia-Pedro von der Santos,✅
...,...,...
95,Ana Clara dô Ribeirô,❌
96,Maria von dêr Mêndês,❌
97,Ênio von dêr Souza,❌
98,Ênio Costà,✅


#### 1.1.3. Add identifiers to dict

In [ ]:

# 5️⃣ Add the subkeys: "Field_Identifier", "Content_Identifier", and "DLP_Strategy"
icu_data_dict_granular["Patient_Identifiers"]["Fields"]["Complete_Name"]["Field_Identifier"] = [
    r"\b(?:[Nn]ome)\b",
    r"[nN]ome\s{0,3}[cC]omplete"
    r"\b[Nn]ome\s{0,3}(?:d[oa])?\s{0,3}[Pp]aciente\b",
    r"[Pp]aciente"]

icu_data_dict_granular["Patient_Identifiers"]["Fields"]["Complete_Name"]["Content_Identifier"] = r"\b[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3}-?(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\bvon der\b|\bvan der\b|\bvan den\b|du|\bde la\b|al|bin|ibn|el)?\s{0,3}[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3}-?(?:(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\bvon der\b|\bvan der\b|\bvan den\b|du|\bde la\b|al|bin|ibn|el)?\s{0,3}-?[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3})*\b"

icu_data_dict_granular["Patient_Identifiers"]["Fields"]["Complete_Name"]["DLP_Strategy"] = "Detects full names using regex-based entity recognition."

### 1.2. Name and Surname (partial Matches)

##### 1.2.1. Build Patterns

In [ ]:
# Field can be either "Nome" (First Name) or "Sobrenome" (Last Name), it is also possible. In Brazil we do not say middle-names, we just call all of them surnames "sobrenomes", so we pluralize.
Field_Identifier = [r"\b(?:[Nn]ome)\b",r"\b[Pp]rimeiro\s{0,3}[nN]ome", r"\b[Ss]obrenomes?\b"]

# Regex for single names or single surnames, allowing for hyphenated names
Content_Identifier = r"\b(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\bvon der\b|\bvan der\b|\bvan den\b|du|de la|al|bin|ibn|el)?\s{0,3}[A-ZÀ-Ÿ][a-zà-ÿ]+\s{0,3}-?(?:[A-ZÀ-Ÿ][a-zà-ÿ]+)?\b"


##### 1.2.2. Stress-Test

In [48]:
test_names = ["João", "Fernanda", "Ana-Clara", "Silva", "Mendes", "dos Santos", "Medeiros", "von Dorf","van Dorf", "João das Marias"]
name_regex_test = re.compile(Content_Identifier)

# Test regex against generated names
matches_test = [name for name in test_names if name_regex_test.fullmatch(name)]

# Display results
df_results_test = pd.DataFrame({"Generated Name": test_names, "Matched": ["✅" if name in matches_test else "❌" for name in test_names]})
df_results_test

,Generated Name,Matched
0,João,✅
1,Fernanda,✅
2,Ana-Clara,✅
3,Silva,✅
4,Mendes,✅
5,dos Santos,✅
6,Medeiros,✅
7,von Dorf,✅
8,van Dorf,✅
9,João das Marias,❌


##### 1.2.3. Add Identifiers to Dict

In [59]:
icu_data_dict_granular["Patient_Identifiers"]["Fields"]["Name_or_Surname"] = {
    "Field_Identifier": Field_Identifier,
    "Content_Identifier": Content_Identifier,
    "DLP_Strategy": "Detects individual names or surnames when stored in separate fields."
}


##### 1.3. Build Patterns

In [ ]:
# Field variations for "Número de Atendimento"
#There are many ways to abbreviate number (N./n./nº/#)
# Alternative words for número could be used, as "ID", "identificação"
# Inserted a type of idiomatic variation which is inverting the words, instead of "número de atendimento" saying "atendimento número"
Field_Identifier = [
    r"\b(?:n[úu]mero|N\.?|#|Nº|I[Dd]|C[óo]digo|identifica(?:ção|cao))\s{0,3}(?:de)?\s{0,3}atendimento\b",
    r"\batendimento\s{0,3}(?:n[úu]mero|N\.?|#|Nº|I[Dd]|C[óo]digo|identifica(?:ção|cao))\b"
]

# Patient ID is usually a 5 to 10-digit number
Content_Identifier = r"\b\d{5,10}\b"


##### 1.2. Stress-Test

In [56]:
test_patient_ids = ["12345", "9876543210", "00001", "ABCDE"]
id_regex_test = re.compile(Content_Identifier)

# Test regex against generated IDs
matches_test = [pid for pid in test_patient_ids if id_regex_test.fullmatch(pid)]

# Display results
df_results_test = pd.DataFrame({"Generated ID": test_patient_ids, "Matched": ["✅" if pid in matches_test else "❌" for pid in test_patient_ids]})
df_results_test

,Generated ID,Matched
0,12345,✅
1,9876543210,✅
2,00001,✅
3,ABCDE,❌


##### 1.3. Add Identifiers to Dict

In [ ]:
icu_data_dict_granular["Patient_Identifiers"]["Fields"]["Name_or_Surname"] = {
    "Field_Identifier": Field_Identifier,
    "Content_Identifier": Content_Identifier,
    "DLP_Strategy": "Detects individual names or surnames when stored in separate fields."
}


##### .2. Build Patterns

In [ ]:
# Field variations for "Nº Prontuário"
Field_Identifier = [r"\b(?:Nº Prontuário|Número do Prontuário)\b"]

# Prontuário numbers are usually between 5 and 10 digits
Content_Identifier = r"\b\d{5,10}\b"


##### 1.2. Stress-Test

In [ ]:
test_record_numbers = ["54321", "1234567890", "98765", "A123B"]
record_regex_test = re.compile(Content_Identifier)

# Test regex against generated record numbers
matches_test = [rec for rec in test_record_numbers if record_regex_test.fullmatch(rec)]

# Display results
df_results_test = pd.DataFrame({"Generated Record Number": test_record_numbers, "Matched": ["✅" if rec in matches_test else "❌" for rec in test_record_numbers]})
tools.display_dataframe_to_user(name="Nº Prontuário Regex Test Results", dataframe=df_results_test)


##### 1.3. Add Identifiers to Dict

In [ ]:
"Field_Regex": [r"\b(?:n[úu]mero|N\.?|#|Nº|I[Dd]|C[óo]digo|identifica(?:ção|cao))\s{0,3}(?:de)?\s{0,3}atendimento\b",
                r"\batendimento\s{0,3}(?:n[úu]mero|N\.?|#|Nº|I[Dd]|C[óo]digo|identifica(?:ção|cao))\b"], 
"Content_Regex": r"\b\d{5,10}\b",
"DLP_Strategy": "Detects numerical patient identifiers."

# Clinical Medical Data

## Diagnosis

Build Pattern

In [ ]:
#Sepsis is one of the most common admission causes to the ICU.
# the word Sepsis comes from σῆψις in old greek, in portuguese it could have the following variations:
# 1. Sepse, sepsis (latin variation), séptico, septicemia. 
# 2. Sepsis nowadays is also described as generalized infection.
# Additionaly, I have done research on the most common admission causes to ICU.



Guide to nurses: https://www.freshrn.com/most-common-icu-admission-diagnosis/ <br>
Pubmed publication: https://pmc.ncbi.nlm.nih.gov/articles/PMC8889595/

In [ ]:

Field_Regex = r"\b(?:Diagn[óo]stico|\bCID-?10\b|Doen[çc]a|Enfermidade|Patologia|Condi[çc][ãa]o|S[ií]ndrome|Achado Clínico|Comorbidade|Progn[óo]stico|Classifica[çc][ãa]o|Quadro\s{0,3}Clínico)\b",
Content_Regex = (
    r"(?:"
    r"S[eé]p(?:tico|ticemia|sis|se)s?|"
    r"neuro|cereb(?:ral|ro)|card|"
    r"infec[cç][aã]o|"
    r"Pneumonia|"
    r"Hemorragia"
    r"Síndrome|"
    r"Hipertens[ãa][oae]|"
    r"Diabetes|"
    r"AVC|aneurisma|Enfart[eo]|"
    r"\bInsufici[êe]ncia\s{0,3}Renal\b|"
    r"\bFal[êe]ncia\s{0,3}Hep[áa]tica\b|"
    r"\bLes[ãa]o\s{0,3}Renal\s{0,3}Aguda\b|"
    r"\bFal[êe]ncia\s{0,3}(?:de M[úu]ltiplos|múltipla de)\s{0,3}[Óo]rg[ãa]os\b|"
    r"\bEmbolia\s{0,3}Pulmonar\b|"
    r"\bInsufici[êe]ncia\s{0,3}Respirat[óo]ria\b|"
    r"\bFal[êe]ncia\s{0,3}Card[íi]aca\b|"
    r"\bParada\s{0,3}Card[íi]aca\b"
    r")"
)


### Positive Stress-Test (PST)

In [50]:
pst_cases = [
    "Paciente com sepse grave", "Diagnóstico: Pneumonia adquirida na comunidade",
    "Infecção generalizada", "CID-10: I10 - Hipertensão essencial",
    "Aneurisma cerebral", "Insuficiência renal aguda",
    "Parada cardiorrespiratória", "Falência múltipla de órgãos",
    "Embolia pulmonar maciça", "Doença cardíaca isquêmica",
    "Insuficiência respiratória grave", "Comorbidade de diabetes e hipertensão",
    "Paciente com AVC hemorrágico", "Síndrome metabólica avançada",
    "Diagnóstico confirmado de encefalopatia hepática",
    "Lesão renal aguda devido a sepse", "Septicemia neonatal",
    "Hipertensão pulmonar secundária", "Falência hepática fulminante",
    "Infecção urinária complicada", "Paciente com quadro de choque séptico",
    "Diagnóstico de cardiomiopatia dilatada", "Edema agudo de pulmão",
    "Hemorragia intracraniana", "Paciente apresenta falência respiratória",
    "História de insuficiência cardíaca descompensada",
    "Paciente diabético descompensado", "Doença pulmonar obstrutiva crônica",
    "Infecção pós-operatória grave", "Paciente internado por choque cardiogênico",
    "Hipotensão refratária", "Paciente apresenta pneumonia nosocomial",
    "Embolia cerebral", "Síndrome do desconforto respiratório agudo",
    "Complicação de insuficiência hepática", "Paciente com diagnóstico de pericardite",
    "Parada cardíaca revertida", "Infarto agudo do miocárdio",
    "Diagnóstico diferencial entre encefalopatia e meningite",
    "Paciente com hipertensão maligna", "Paciente internado com insuficiência renal terminal",
    "Doença pulmonar intersticial avançada", "Paciente em coma hepático",
    "História de arritmia ventricular complexa", "Paciente com crise hipertensiva",
    "Diagnóstico de insuficiência cardíaca congestiva",
    "Infecção de corrente sanguínea associada a cateter", "Paciente com quadro de meningite bacteriana",
    "Paciente com falência de múltiplos órgãos após trauma",
]


# Apply regex
pst_results = ["✅ Matched" if re.search(Content_Regex, case, re.IGNORECASE) else "❌ Missed" for case in pst_cases]

# Display results
df_pst_results = pd.DataFrame({"PST Test Case": pst_cases, "Detection Result": pst_results})
df_pst_results


,PST Test Case,Detection Result
0,Paciente com sepse grave,✅ Matched
1,Diagnóstico: Pneumonia adquirida na comunidade,✅ Matched
2,Infecção generalizada,✅ Matched
3,CID-10: I10 - Hipertensão essencial,✅ Matched
4,Aneurisma cerebral,✅ Matched
5,Insuficiência renal aguda,✅ Matched
6,Parada cardiorrespiratória,✅ Matched
7,Falência múltipla de órgãos,✅ Matched
8,Embolia pulmonar maciça,✅ Matched
9,Doença cardíaca isquêmica,✅ Matched


## Negative Stress-Test(NST)

In [51]:
nst_cases = [
    "Paciente apresentou melhora clínica", "Alta hospitalar programada para amanhã",
    "Tomografia do crânio sem alterações significativas",
    "Paciente sem histórico de doenças crônicas", "Necessidade de ventilação mecânica avaliada",
    "Discussão sobre novas diretrizes para sepse", "Estudos indicam aumento da resistência bacteriana",
    "Artigo científico sobre embolia pulmonar", "Hospital adquiriu novos monitores cardíacos",
    "Paciente transferido para unidade de cuidados intermediários",
    "Técnicas avançadas de suporte ventilatório", "Não há sinais de infecção ativa",
    "Discussão de casos clínicos sobre hipertensão", "Histórico de infecções prévias analisado",
    "Paciente encaminhado para fisioterapia respiratória",
    "Enfermagem monitora sinais vitais regularmente", "Paciente será submetido a cirurgia eletiva",
    "Evolução clínica satisfatória após antibiótico", "Paciente sem necessidade de diálise no momento",
    "Nova pesquisa sobre falência cardíaca em idosos", "Medicação ajustada conforme evolução do quadro",
    "Paciente acordado e orientado, sem déficits neurológicos",
    "Avaliação de risco para embolia pulmonar sem achados clínicos",
    "Nenhuma alteração no eletrocardiograma do paciente",
    "Necessidade de internação hospitalar reavaliada",
    "Paciente sem queixas de dor torácica", "Protocolo de prevenção de infecções hospitalares atualizado",
    "Treinamento da equipe médica sobre sepse", "Não há sinais de agravamento da função hepática",
    "Discussão sobre manejo clínico da hipertensão", "Paciente sem necessidade de intervenção cirúrgica",
    "Paciente será avaliado para alta da UTI", "Nenhuma evidência de sepse ou infecção",
    "Resultados laboratoriais sem alterações relevantes",
    "Conferência médica discute tratamentos inovadores",
    "Paciente permanece hemodinamicamente estável",
    "Análise estatística sobre taxas de internação hospitalar",
    "Paciente sem sinais de desconforto respiratório",
    "Análise retrospectiva de casos de insuficiência cardíaca",
    "Paciente apresenta boa resposta ao tratamento",
    "Discussão sobre novas diretrizes para choque séptico",
    "Equipe médica realiza estudo comparativo sobre pneumonia",
    "Paciente sem queixas e com sinais vitais normais",
    "Avaliação médica descarta necessidade de ventilação mecânica",
    "Progresso do paciente avaliado como satisfatório",
    "Pesquisa clínica sobre fatores de risco para diabetes",
    "Resultados de exames descartam patologia grave",
    "Paciente recebe alta após monitoramento na UTI",
]


nst_results = ["❌ Correctly Ignored" if not re.search(Content_Regex, case, re.IGNORECASE) else "⚠️ FALSE POSITIVE" for case in nst_cases]
df_nst_results = pd.DataFrame({"NST Test Case": nst_cases[:100], "Detection Result": nst_results[:100]})
df_nst_results.style

,NST Test Case,Detection Result
0,Paciente apresentou melhora clínica,❌ Correctly Ignored
1,Alta hospitalar programada para amanhã,❌ Correctly Ignored
2,Tomografia do crânio sem alterações significativas,❌ Correctly Ignored
3,Paciente sem histórico de doenças crônicas,❌ Correctly Ignored
4,Necessidade de ventilação mecânica avaliada,❌ Correctly Ignored
5,Discussão sobre novas diretrizes para sepse,⚠️ FALSE POSITIVE
6,Estudos indicam aumento da resistência bacteriana,❌ Correctly Ignored
7,Artigo científico sobre embolia pulmonar,⚠️ FALSE POSITIVE
8,Hospital adquiriu novos monitores cardíacos,⚠️ FALSE POSITIVE
9,Paciente transferido para unidade de cuidados intermediários,❌ Correctly Ignored


In [55]:
mst_cases = [
    # 1️⃣ Contextual Evasion
    "O exame revelou um quadro típico de Sepse, segundo a análise médica.",
    "Foi prescrito um tratamento emergencial devido ao risco de parada cardíaca.",

    # 2️⃣ Structured Data Evasion
    "Paciente: João \n Diagnóstico:\n\n Sepse severa.",
    "Nome: João | Diagnóstico: AVC | Tratamento: Heparina",
    "Paciente-Estado: ParadaCardiaca",

    # 3️⃣ Semantic Substitution
    "O paciente apresentou um quadro clínico compatível com choque séptico.",
    "Hipotensão grave associada a infecção sistêmica.",

    # 4️⃣ Encoding Tricks
    "Pneumonia".encode('utf-8').hex(),  # Stored as hex encoding
    "P%6E%65umonia",  # URL encoding attempt
    bytes("Pneumonia", "utf-8").decode("unicode_escape"),  # Unicode escape sequences
    "Pneu\x6dm\x6eia",  # Hex-like obfuscation
    "Pneum&#111;nia"  # HTML entity encoding attempt
]

# Apply regex
mst_results = ["✅ Matched" if re.search(Content_Regex, case, re.IGNORECASE) else "❌ Evaded" for case in mst_cases]

# Display results
df_mst_results = pd.DataFrame({"MST Test Case": mst_cases, "Detection Result": mst_results})
df_mst_results

df_mst_results


,MST Test Case,Detection Result
0,"O exame revelou um quadro típico de Sepse, seg...",✅ Matched
1,Foi prescrito um tratamento emergencial devido...,✅ Matched
2,Paciente: João \n Diagnóstico:\n\n Sepse severa.,✅ Matched
3,Nome: João | Diagnóstico: AVC | Tratamento: He...,✅ Matched
4,Paciente-Estado: ParadaCardiaca,✅ Matched
5,O paciente apresentou um quadro clínico compat...,✅ Matched
6,Hipotensão grave associada a infecção sistêmica.,✅ Matched
7,506e65756d6f6e6961,❌ Evaded
8,P%6E%65umonia,❌ Evaded
9,Pneumonia,✅ Matched


## Time Stress-Test (TST)

### **⏳ TST Goals:**
- **Detect linguistic drift** – Identify when new terms appear or old ones become obsolete.
- **Validate long-term robustness** – Ensure the regex captures relevant diagnoses over time.
- **Trigger review alerts** – Raise warnings if medical dictionaries update with new critical conditions.

### **⚙️ TST Implementation:**
- **Automated Monitoring** → Periodically compare regex matches against **ICD-10, SNOMED-CT, and real ICU records** to detect gaps.
- **Historical Tracking** → Log **term frequency** in hospital datasets to monitor shifts in **ICU admission causes**.
- **Adaptability Index** → Classify identifiers based on their **likelihood of change**, setting **periodic review cycles**.


## Medical Prescription

#### Build pattern

In [60]:
## Medical Prescription

Field_Regex = r"\b(?:Prescriç[ãa]o|Receita M[ée]dica|Medicação|Drogas|Farmacologia|Medicamento|Uso Contínuo|Administração)\b"

Content_Regex = (
    r"\b(?:"
    # 🔹 **Analgésicos e Sedativos**
    r"Paracetamol|Dipirona|Ibuprofeno|Cetoprofeno|Tramadol|"
    r"Morfina|Fentanil|Sufentanil|Meperidina|Clonidina|"
    r"Midazolam|Lorazepam|Diazepam|Propofol|Ketamina|Dexmedetomidina|"

    # 🔹 **Vasopressores e Inotrópicos**
    r"Dobutamina|Dopamina|Adrenalina|Noradrenalina|Vasopressina|"
    r"Milrinona|Fenilefrina|"

    # 🔹 **Antibióticos e Antifúngicos**
    r"Vancomicina|Meropenem|Imipenem|Piperacilina|Tazobactam|"
    r"Cefepima|Ceftriaxona|Ceftazidima|Levofloxacino|Ciprofloxacino|"
    r"Amicacina|Gentamicina|Linezolida|Fluconazol|Anfotericina|"

    # 🔹 **Cardiovasculares**
    r"Amiodarona|Labetalol|Metoprolol|Esmolol|Nicardipina|"
    r"Hidralazina|Nitroprussiato|Diltiazem|"

    # 🔹 **Antipsicóticos e Neuromusculares**
    r"Haloperidol|Quetiapina|Olanzapina|Risperidona|"
    r"Pancurônio|Rocurônio|Cisatracúrio|Atracúrio|"

    # 🔹 **Outros Medicamentos de Uso Comum**
    r"Atorvastatina|Omeprazol|Pantoprazol|Ranitidina|"
    r"Heparina|Enoxaparina|"

    r")\b"
)




# --- 3️⃣ POSITIVE STRESS TEST (PST) ---


In [61]:

pst_cases = [
    "Paciente em uso de Morfina para controle da dor.",
    "Prescrição de Paracetamol a cada 6 horas.",
    "Fentanil foi administrado em dose baixa.",
    "Dipirona e Ibuprofeno prescritos para febre.",
    "Paciente recebeu Midazolam para sedação profunda.",
    "Noradrenalina iniciada para suporte hemodinâmico.",
    "Antibiótico Vancomicina foi incluído na prescrição.",
    "Anfotericina indicada para tratar infecção fúngica.",
    "Hidralazina utilizada para controle da hipertensão.",
    "Dopamina administrada em infusão contínua.",
    "Quetiapina prescrita para agitação psicomotora.",
    "Rocurônio usado para indução anestésica.",
    "Paciente está tomando Omeprazol para proteção gástrica.",
    "Heparina foi administrada para evitar trombose.",
    "Metoprolol utilizado no controle da pressão arterial.",
    "Paciente recebe Enoxaparina como profilaxia antitrombótica.",
    "Labetalol foi prescrito para tratamento de crise hipertensiva.",
    "Midazolam foi administrado em protocolo de sedação contínua.",
    "Paciente apresentou boa resposta ao tratamento com Atorvastatina.",
    "Cefepima foi escolhida para tratamento de pneumonia grave."
]

# Apply regex
pst_results = ["✅ Matched" if re.search(Content_Regex, case, re.IGNORECASE) else "❌ Missed" for case in pst_cases]

# Display results
df_pst_results = pd.DataFrame({"PST Test Case": pst_cases, "Detection Result": pst_results})
df_pst_results


,PST Test Case,Detection Result
0,Paciente em uso de Morfina para controle da dor.,✅ Matched
1,Prescrição de Paracetamol a cada 6 horas.,✅ Matched
2,Fentanil foi administrado em dose baixa.,✅ Matched
3,Dipirona e Ibuprofeno prescritos para febre.,✅ Matched
4,Paciente recebeu Midazolam para sedação profunda.,✅ Matched
5,Noradrenalina iniciada para suporte hemodinâmico.,✅ Matched
6,Antibiótico Vancomicina foi incluído na prescr...,✅ Matched
7,Anfotericina indicada para tratar infecção fún...,✅ Matched
8,Hidralazina utilizada para controle da hiperte...,✅ Matched
9,Dopamina administrada em infusão contínua.,✅ Matched


In [ ]:


## Medical Prescription

Field_Regex = r"\b(?:Prescriç[ãa]o|Receita M[ée]dica|Medicação|Drogas|Farmacologia|Medicamento|Uso Contínuo|Administração)\b"

Content_Regex = (
    r"\b(?:"
    # 🔹 **Analgésicos e Sedativos**
    r"Paracetamol|Dipirona|Ibuprofeno|Cetoprofeno|Tramadol|"
    r"Morfina|Fentanil|Sufentanil|Meperidina|Clonidina|"
    r"Midazolam|Lorazepam|Diazepam|Propofol|Ketamina|Dexmedetomidina|"

    # 🔹 **Vasopressores e Inotrópicos**
    r"Dobutamina|Dopamina|Adrenalina|Noradrenalina|Vasopressina|"
    r"Milrinona|Fenilefrina|"

    # 🔹 **Antibióticos e Antifúngicos**
    r"Vancomicina|Meropenem|Imipenem|Piperacilina|Tazobactam|"
    r"Cefepima|Ceftriaxona|Ceftazidima|Levofloxacino|Ciprofloxacino|"
    r"Amicacina|Gentamicina|Linezolida|Fluconazol|Anfotericina|"

    # 🔹 **Cardiovasculares**
    r"Amiodarona|Labetalol|Metoprolol|Esmolol|Nicardipina|"
    r"Hidralazina|Nitroprussiato|Diltiazem|"

    # 🔹 **Antipsicóticos e Neuromusculares**
    r"Haloperidol|Quetiapina|Olanzapina|Risperidona|"
    r"Pancurônio|Rocurônio|Cisatracúrio|Atracúrio|"

    # 🔹 **Outros Medicamentos de Uso Comum**
    r"Atorvastatina|Omeprazol|Pantoprazol|Ranitidina|"
    r"Heparina|Enoxaparina|"

    r")\b"
)

# --- 4️⃣ NEGATIVE STRESS TEST (NST) ---
nst_cases = [
    "O hospital recebeu uma nova remessa de seringas.",
    "Equipe de enfermagem realizou treinamento em cuidados intensivos.",
    "Lista de equipamentos disponíveis no setor de emergência.",
    "Reunião com fornecedores para aquisição de novos insumos.",
    "Procedimentos administrativos para solicitação de medicamentos.",
    "Paciente não apresentou necessidade de medicação adicional.",
    "A farmácia hospitalar organiza os estoques semanalmente.",
    "Discussão sobre melhores práticas em prescrição médica.",
    "Professor de farmacologia apresenta nova pesquisa.",
    "Relatório sobre uso racional de medicamentos na UTI.",
    "Insumos médicos estão disponíveis para enfermagem.",
    "Política de segurança hospitalar reforçada.",
    "Gestão hospitalar planeja novas estratégias para atendimento.",
    "Pesquisa avalia eficácia de tratamento não farmacológico.",
    "Artigo publicado sobre tendências em terapias médicas.",
    "Especialistas discutem avanços na área de anestesiologia.",
    "A farmácia clínica é essencial para otimizar prescrições.",
    "Engenharia hospitalar realiza manutenção em ventiladores.",
    "Comissão de controle de infecção analisa novos protocolos.",
    "Hospital inaugura novo laboratório de análises clínicas."
]

# Apply regex
nst_results = ["✅ Matched" if re.search(Content_Regex, case, re.IGNORECASE) else "❌ Correctly Ignored" for case in nst_cases]

# Display results
df_nst_results = pd.DataFrame({"NST Test Case": nst_cases, "Detection Result": nst_results})
df_nst_results.style

,NST Test Case,Detection Result
0,O hospital recebeu uma nova remessa de seringas.,✅ Matched
1,Equipe de enfermagem realizou treinamento em cuidados intensivos.,✅ Matched
2,Lista de equipamentos disponíveis no setor de emergência.,✅ Matched
3,Reunião com fornecedores para aquisição de novos insumos.,✅ Matched
4,Procedimentos administrativos para solicitação de medicamentos.,✅ Matched
5,Paciente não apresentou necessidade de medicação adicional.,✅ Matched
6,A farmácia hospitalar organiza os estoques semanalmente.,✅ Matched
7,Discussão sobre melhores práticas em prescrição médica.,✅ Matched
8,Professor de farmacologia apresenta nova pesquisa.,✅ Matched
9,Relatório sobre uso racional de medicamentos na UTI.,✅ Matched


In [ ]:


## **Build Pattern**
icu_data_dict_granular["Clinical_Medical_Data"]["Fields"]["Diagnósticos"] = {
    "Field_Regex": r"\b(?:Diagnóstico|CID-10|Doença)\b",
    "Content_Regex": r"(?:Sepsis|Pneumonia|Hipertensão|Diabetes)",
    "DLP_Strategy": "Uses NLP-based classification to recognize medical conditions."
}

## **Stress Test**
medical_conditions = ["Sepsis", "Diabetes", "Hipertensão", "Pneumonia"]
condition_regex = re.compile(icu_data_dict_granular["Clinical_Medical_Data"]["Fields"]["Diagnósticos"]["Content_Regex"])
matches = [cond for cond in medical_conditions if condition_regex.fullmatch(cond)]
df_results = pd.DataFrame({"Generated Medical Conditions": medical_conditions, "Matched": ["✅" if cond in matches else "❌" for cond in medical_conditions]})
tools.display_dataframe_to_user(name="Medical Condition Regex Test", dataframe=df_results)

# 6️⃣ Medication & Treatment
icu_data_dict_granular["Medication_Treatment"] = {"Fields": {}}

## **Build Pattern**
icu_data_dict_granular["Medication_Treatment"]["Fields"]["Prescrição"] = {
    "Field_Regex": r"\b(?:Prescrição|Receita Médica)\b",
    "Content_Regex": r"\b(?:Paracetamol|Ibuprofeno|Morfina|Fentanil)\b",
    "DLP_Strategy": "Flags prescription of controlled substances."
}

# 7️⃣ Final Steps: Export Dictionary
with open("icu_dlp_dictionary.json", "w", encoding="utf-8") as f:
    json.dump(icu_data_dict_granular, f, indent=4, ensure_ascii=False)

print("ICU DLP Dictionary saved as JSON ✅")
